In [1]:
import datetime
import time
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import SGD
from sklearn.preprocessing import MinMaxScaler
from keras import metrics


from statsmodels.compat.pandas import deprecate_kwarg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
# additive decompose a contrived additive time series
from random import randrange
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose 

# the main library has a small set of functionality
from stldecompose import decompose, forecast
from stldecompose.forecast_funcs import (naive,
                                         drift, 
                                         mean, 
                                         seasonal_naive)
%load_ext autoreload
%autoreload 2

# funzioni per preparare i dati e calcolare le previsioni
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
         # find the end of this pattern
         end_ix = i + n_steps_in
         out_end_ix = end_ix + n_steps_out
         # check if we are beyond the sequence
         if out_end_ix > len(sequence):
             break
         # gather input and output parts of the pattern
         seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
         X.append(seq_x)
         y.append(seq_y)
    return np.array(X), np.array(y)


def get_forcast_per_component(series, st_in, st_out, train_test_size):
    
    # split into samples
    X, y = split_sequence(series, st_in, st_out)

    train_X, test_X = X[:train_test_size], X[train_test_size:]
    train_y, test_y = y[:train_test_size], y[train_test_size:]

    # define model
    model = Sequential()
    model.add(Dense(500, activation='relu', input_dim=st_in)) 
    model.add(Dense(st_out))

    model.compile(optimizer='adam', loss='mse' , metrics=[metrics.mae, 'accuracy'])

    # fit model
    model.fit(train_X, train_y, batch_size=64, epochs=200, verbose=0)
    
    # predict 
    predicted = []
    for i in range(len(test_X)):
        x_input = test_X[i].reshape(1, st_in)
        yhat = model.predict(x_input, verbose=0)
        
        #predicted.append(np.rint(yhat[0]))   
        predicted.append(np.around(yhat[0], decimals=1)) 
    predicted = np.array(predicted)
    return predicted


# carica dati
data = pd.read_csv('/Users/alket/Desktop/dati/new_data_backfill_forwfill.csv', index_col = 0, 
                   header=0, parse_dates=True)

# aggrega dati
agg_by_cell = data.groupby(by = ['cell_num'])

# dichiara counter e struttura dati per i dati d'errore per cella
counter = 0
dict2data = {}
dict2MAPE = {}
# ittera per tutte le celle
for ii, kk in agg_by_cell:
    # metti i dati nel formatto giusto
    cell = ii
    error_list = []
    print(counter)
    counter +=1
    #if counter > 17: break
    dates4dec = []
    cell_values = []

    for index, row in kk.iterrows():
    
        date = row['date']
        h = str(row['hours'])
   
        h = h.split('.')
    
        if len(h[0])<2:
            h = h[1]+h[0]
        else: 
            h = h[0]
   
        minutes = str(row['minutes'])
        m = ''
        minutes = minutes.split('.')
        if len(minutes[0]) < 2: 
            m = minutes[0] +'0'
        else: 
            m = minutes[0]
        #print(date, h, m)
        data_f = date+' '+h+':'+m+':'+'00'
        #print(data_f)
        cell_values.append(row['nr_people'])
        dates4dec.append(data_f) 


    dict_i = {'ds': dates4dec, 'y':cell_values}
    data4deco = pd.DataFrame(dict_i, index=None, columns=None)  
    data4deco.head()

    data4deco['ds'] = pd.to_datetime(data4deco['ds'])
    data4deco = data4deco.set_index('ds')
    data4deco.head()


    # decomponi i dati in trend, residual e seasonal
    decomp = decompose(data4deco['y'], period=96)

    # visualizza grafico : da commentare per non consumare memoria
    #with plt.rc_context():
    #     plt.rc("figure", figsize=(18,10))
    #     decomp.plot()
    #     plt.show()

    trend = decomp.trend.values
    seasonal = decomp.seasonal.values
    residual = decomp.resid.values

    # imposta step di previsione e chiama funzione get_forcast_per_component su ogni componente
    n_steps_in, n_steps_out = 26, 16
    train_test_size = 9000

    forcasted_trend = get_forcast_per_component(trend, n_steps_in, n_steps_out, train_test_size)
    forcasted_residual = get_forcast_per_component(residual, n_steps_in, n_steps_out, train_test_size)
    forcasted_season = get_forcast_per_component(seasonal, n_steps_in, n_steps_out, train_test_size)

    # combina le previsioni 
    final_prediction = forcasted_trend + forcasted_residual + forcasted_season

    # fai lo split del serie dati originale
    X, y = split_sequence(kk['nr_people'].values, n_steps_in, n_steps_out) 

    # prepare train-test della serie originale
    train_X, test_y = X[:train_test_size], X[train_test_size:]
    train_y, test_y = y[:train_test_size], y[train_test_size:]

    # assegna a expected il valore del test set
    expected = test_y

    # calcola differenza (errore) tra predicted e expected 
    difference = abs(expected - final_prediction)
    MAPE = np.mean(abs(100 * (difference/expected)))
    dict2MAPE[cell] = MAPE
    # calcola errore medio e altre misure 
    mean_error =  np.reshape(difference, difference.shape[0] * difference.shape[1])
    print('Mean error', np.mean(mean_error))
    
    # collect data 2 dictionary
    minimum = np.amin(mean_error)   
    per75 = np.percentile(mean_error, 75)
    per50 = np.percentile(mean_error, 50)
    per25 = np.percentile(mean_error, 25)
    maximum = np.amax(mean_error)
    l5i = [minimum, per25, per50, per75, maximum]
    dict2data[cell] = l5i

    
with open('error_data_4_MLP_with_STL_Decomposition_in26_out16_period_96.csv', 'w') as f:
    for key, value in dict2data.items():
        f.write('%s:%s\n' % (key, value))  
              

Using TensorFlow backend.


0
Mean error 2.103812827760664
1
Mean error 2.270396638829914
2
Mean error 2.4521006661543656
3
Mean error 2.5900570479032146
4
Mean error 2.5348459686404063
5
Mean error 1.9881211796692662
6
Mean error 1.7897633377679385
7
Mean error 1.7762536317120832
8
Mean error 1.7501219990971204
9
Mean error 1.7972601691565877
10
Mean error 1.7033497802522755
11
Mean error 18.352100479848385
12
Mean error 18.840687786925344
13
Mean error 14.099489467773878
14
Mean error 4.8797479229021645
15
Mean error 0.9419565275338396
16
Mean error 1.0210832857594492
17
Mean error 2.140461236507745
18
Mean error 2.631396836953599
19
Mean error 2.4977253218355444
20
Mean error 2.6705254409388166
21
Mean error 2.5485091641098316
22
Mean error 2.5635885792946964
23
Mean error 2.0260524936303104
24
Mean error 1.8489361914392408
25
Mean error 1.817500937726035
26
Mean error 1.761817866514252
27
Mean error 2.970091729936162
28
Mean error 21.45316001366828
29
Mean error 16.890698420169944
30
Mean error 15.80156081240

In [3]:
with open('MAPE_data_4_MLP_with_STL_Decomposition_in26_out16_period_96.csv', 'w') as f:
    for key, value in dict2MAPE.items():
        f.write('%s:%s\n' % (key, value)) 